# Custom Estimator with Keras

**Learning Objectives**
- Learn how to create custom estimator using tf.keras
    
Up until now we've been limited in our model architectures to premade estimators. But what if we want more control over the model? 

We can use the popular Keras API to create a custom model and then convert it to an estimator using `tf.keras.estimator.model_to_estimator()`. 

This gives us access to all the flexibility of Keras for creating deep learning models, but also the production readiness of the estimator framework!

In [1]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


## Train and Evaluate Input Functions

Same as before

In [2]:
CSV_COLUMN_NAMES = ["fare_amount","dayofweek","hourofday","pickuplon","pickuplat","dropofflon","dropofflat"]
CSV_DEFAULTS = [[0.0],[1],[0],[-74.0], [40.0], [-74.0], [40.7]]

def read_dataset(csv_path):
    def parse_row(row):
        # Decode the CSV row into list of TF tensors
        fields = tf.decode_csv(records = row, record_defaults = CSV_DEFAULTS)

        # Pack the result into a dictionary
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        
        # NEW: Add engineered features
        features = add_engineered_features(features)
        
        # Separate the label from the features
        label = features.pop("fare_amount") # remove label from features and store

        return features, label
    
    # Create a dataset containing the text lines.
    dataset = tf.data.Dataset.list_files(file_pattern = csv_path) # (i.e. data_file_*.csv)
    dataset = dataset.flat_map(map_func = lambda filename: tf.data.TextLineDataset(filenames = filename).skip(count = 1))

    # Parse each CSV row into correct (features,label) format for Estimator API
    dataset = dataset.map(map_func = parse_row)
    
    return dataset
  
def create_feature_keras_input(features, label):
  features = tf.feature_column.input_layer(features = features, feature_columns = create_feature_columns())
  return features, label

def train_input_fn(csv_path, batch_size = 128):
    #1. Convert CSV into tf.data.Dataset with (features, label) format
    dataset = read_dataset(csv_path)
      
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(buffer_size = 1000).repeat(count = None).batch(batch_size = batch_size)
    
    #3. Create single feature tensor for input to Keras Model
    dataset = dataset.map(map_func = create_feature_keras_input)
   
    return dataset

def eval_input_fn(csv_path, batch_size = 128):
    #1. Convert CSV into tf.data.Dataset with (features, label) format
    dataset = read_dataset(csv_path)

    #2.Batch the examples.
    dataset = dataset.batch(batch_size = batch_size)
    
    #3. Create single feature tensor for input to Keras Model
    dataset = dataset.map(map_func = create_feature_keras_input)
   
    return dataset

## Feature Engineering

Same as before

In [3]:
def add_engineered_features(features):
    features["latdiff"] = features["pickuplat"] - features["dropofflat"] # East/West
    features["londiff"] = features["pickuplon"] - features["dropofflon"] # North/South
    features["euclidean_dist"] = tf.sqrt(x = features["latdiff"]**2 + features["londiff"]**2)

    return features

In [4]:
def create_feature_columns():
  # One hot encode dayofweek and hourofday
  fc_dayofweek = tf.feature_column.categorical_column_with_identity(key = "dayofweek", num_buckets = 8)
  fc_hourofday = tf.feature_column.categorical_column_with_identity(key = "hourofday", num_buckets = 24)

  # Cross features to get combination of day and hour
  fc_day_hr = tf.feature_column.crossed_column(keys = [fc_dayofweek, fc_hourofday], hash_bucket_size = 24 * 7)

  # Bucketize latitudes and longitudes
  NBUCKETS = 16
  latbuckets = np.linspace(38.0, 42.0, NBUCKETS).tolist()
  lonbuckets = np.linspace(-76.0, -72.0, NBUCKETS).tolist()
  fc_bucketized_plat = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "pickuplon"), boundaries = lonbuckets)
  fc_bucketized_plon = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "pickuplat"), boundaries = latbuckets)
  fc_bucketized_dlat = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "dropofflon"), boundaries = lonbuckets)
  fc_bucketized_dlon = tf.feature_column.bucketized_column(source_column = tf.feature_column.numeric_column(key = "dropofflat"), boundaries = latbuckets)

  feature_columns = [
    #1. Engineered using tf.feature_column module
    tf.feature_column.indicator_column(categorical_column = fc_day_hr), # 168 columns
    fc_bucketized_plat, # 16 + 1 = 17 columns
    fc_bucketized_plon, # 16 + 1 = 17 columns
    fc_bucketized_dlat, # 16 + 1 = 17 columns
    fc_bucketized_dlon, # 16 + 1 = 17 columns
    #2. Engineered in input functions
    tf.feature_column.numeric_column(key = "latdiff"), # 1 column
    tf.feature_column.numeric_column(key = "londiff"), # 1 column
    tf.feature_column.numeric_column(key = "euclidean_dist") # 1 column
  ]
  
  return feature_columns

Calculate the number of feature columns that will be input to our Keras model

In [5]:
num_feature_columns = 168 + (16 + 1) * 4 + 3
print("num_feature_columns = {}".format(num_feature_columns))

num_feature_columns = 239


## Build Custom Keras Model

Build a Keras model as described [here](https://www.tensorflow.org/guide/keras).

In [6]:
def create_keras_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(units = 64, activation = "relu", input_shape = (num_feature_columns,)))
  model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
  model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
  model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
  model.add(tf.keras.layers.Dense(units = 8, activation = "relu"))
  model.add(tf.keras.layers.Dense(units = 1, activation = None))

  def rmse(y_true, y_pred): # Root Mean Squared Error
    return tf.sqrt(x = tf.reduce_mean(input_tensor = tf.square(x = y_pred - y_true)))

  model.compile(optimizer = tf.train.AdamOptimizer(),
                loss = "mean_squared_error",
                metrics = [rmse])
  
  return model

## Build Custom Keras Model

In [7]:
# Create serving input function
def serving_input_fn():
  feature_placeholders = {
    "dayofweek": tf.placeholder(dtype = tf.int32, shape = [None]),
    "hourofday": tf.placeholder(dtype = tf.int32, shape = [None]),
    "pickuplon": tf.placeholder(dtype = tf.float32, shape = [None]),
    "pickuplat": tf.placeholder(dtype = tf.float32, shape = [None]),
    "dropofflon": tf.placeholder(dtype = tf.float32, shape = [None]),
    "dropofflat": tf.placeholder(dtype = tf.float32, shape = [None]),
  }
  
  features = {key: tensor for key, tensor in feature_placeholders.items()}
    
  return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

## Train and Evaluate

Note the use of `tf.keras.estimator.model_to_estimator` to create our estimator. It takes as arguments the compiled keras model, the OUTDIR, and optionally a `tf.estimator.Runconfig`

In [8]:
from google.datalab.ml import TensorBoard
TensorBoard().start("taxi_trained")

TensorBoard was started successfully with pid 975935. Click here to access it.

975935

In [9]:
def train_and_evaluate(output_dir):
  estimator = tf.keras.estimator.model_to_estimator(
      keras_model = create_keras_model(),
      model_dir = output_dir,
      config = tf.estimator.RunConfig(
            tf_random_seed = 1, # for reproducibility
            save_checkpoints_steps = 100 # checkpoint every N steps
      )
  )

  train_spec=tf.estimator.TrainSpec(
                     input_fn = lambda: train_input_fn(csv_path = "./taxi-train.csv"),
                     max_steps = 500)

  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

  eval_spec=tf.estimator.EvalSpec(
                     input_fn = lambda: eval_input_fn(csv_path = "./taxi-valid.csv"),
                     steps = None,
                     start_delay_secs = 10, # wait at least N seconds before first evaluation (default 120)
                     throttle_secs = 10, # wait at least N seconds before each subsequent evaluation (default 600)
                     exporters = None) # export SavedModel once at the end of training

  tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [10]:
%%time
OUTDIR = "taxi_trained"
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
train_and_evaluate(OUTDIR)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_service': None, '_experimental_distribute': None, '_save_checkpoints_steps': 100, '_train_distribute': None, '_protocol': None, '_device_fn': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_model_dir': 'taxi_trained', '_master': '', '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_eval_distribute': None, '_evaluation_master': '', '_task_type': 'worker', '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_id': 0, '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d73065438>, '_log_step_count_steps': 100}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorfl

## Cleanup

In [11]:
if len(TensorBoard.list()) > 0:
  [TensorBoard().stop(pid)for pid in TensorBoard.list()["pid"]]
else:
  print("No TensorBoard instances to stop")

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License